In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
28*28

784

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [7]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.36110002,  0.29405573,  0.8111279 ,  0.49007148, -0.34562173,
         0.10034452,  0.20798299, -0.28993863,  0.03263366, -0.290082  ]],
      dtype=float32)

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.1191258 , 0.05078565, 0.09125317, 0.10021425, 0.05609184,
        0.14701016, 0.18408212, 0.10048597, 0.09411305, 0.05683801]],
      dtype=float32)

In [11]:
tf.nn.softmax(predictions).numpy().sum()

1.0

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [13]:
loss_fn(y_train[:1], predictions).numpy()

1.9172536

In [14]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
   1/1875 [..............................] - ETA: 4:29 - loss: 2.1054 - accuracy: 0.2500

2022-07-10 10:34:19.836079: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 1s 640us/step - loss: 0.2936 - accuracy: 0.9145
Epoch 2/5
1875/1875 [==============================] - 1s 610us/step - loss: 0.1429 - accuracy: 0.9572
Epoch 3/5
1875/1875 [==============================] - 1s 609us/step - loss: 0.1070 - accuracy: 0.9679
Epoch 4/5
1875/1875 [==============================] - 1s 612us/step - loss: 0.0877 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 1s 612us/step - loss: 0.0735 - accuracy: 0.9769


In [16]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0770 - accuracy: 0.9761 - 172ms/epoch - 549us/step


[0.07701782137155533, 0.9761000275611877]

In [17]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.0124064e-07, 3.5037584e-08, 4.3097106e-07, 1.6583217e-05,
        1.4618867e-10, 1.2587314e-07, 3.2448589e-14, 9.9997973e-01,
        3.8330263e-09, 2.9743983e-06],
       [1.5209192e-08, 4.5849956e-05, 9.9989486e-01, 5.6438967e-05,
        2.6093866e-16, 2.8203906e-06, 2.0031305e-08, 7.2763116e-16,
        1.5849747e-07, 1.6694509e-15],
       [6.4250742e-08, 9.9952126e-01, 3.8005513e-05, 7.2527664e-06,
        2.5407593e-05, 9.4495899e-06, 5.2546407e-06, 1.9058748e-04,
        2.0104495e-04, 1.7548549e-06],
       [9.9984133e-01, 3.3095759e-09, 2.8351915e-05, 5.0473211e-08,
        4.6405808e-07, 1.5465632e-05, 9.6521202e-05, 1.2425230e-05,
        3.8120117e-08, 5.3205158e-06],
       [1.4813491e-05, 1.0273783e-09, 1.3125622e-06, 1.9805094e-08,
        9.9700898e-01, 3.3663443e-07, 3.9081033e-06, 2.0920876e-05,
        3.6578363e-08, 2.9496425e-03]], dtype=float32)>

In [19]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.9.2


In [23]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [25]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [26]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [28]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)
).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [30]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

In [31]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [32]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [40]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_obj(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_accuracy(labels, predictions)

In [41]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_obj(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [42]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
    
  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13730984926223755, Accuracy: 95.99333190917969, Test Loss: 0.058690495789051056, Test Accuracy: 98.0
Epoch 2, Loss: 0.04348410665988922, Accuracy: 98.65166473388672, Test Loss: 0.05748177319765091, Test Accuracy: 98.18000030517578
Epoch 3, Loss: 0.02184980921447277, Accuracy: 99.26000213623047, Test Loss: 0.05664462223649025, Test Accuracy: 98.23999786376953
Epoch 4, Loss: 0.013125906698405743, Accuracy: 99.58666229248047, Test Loss: 0.06247711181640625, Test Accuracy: 98.33999633789062
Epoch 5, Loss: 0.009413538500666618, Accuracy: 99.69666290283203, Test Loss: 0.059915754944086075, Test Accuracy: 98.43999481201172
